TP1

Laboratorio de Datos

1er Cuatrimestre - 2024

Iván Exequiel Pintos - Joaquín Rovner - Juan José García Vizioli

- Importamos todas las librerías que venimos usando:

In [2]:
import seaborn as sns
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from scipy import stats
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
#from formulaic import model_matrix

- Leemos los dataset:

In [4]:
#@title Usando Colab

from google.colab import drive
drive.mount('/content/drive')

sube_2023_file_path = '/content/drive/My Drive/LDD-TP1/sube-2023.csv'
sube_2023_regresion_file_path = '/content/drive/My Drive/LDD-TP1/sube-2023-regresion.csv'

Mounted at /content/drive


In [ ]:
#@title Usando Jupyter

sube_2023_file_path = '../LDD-TP1/sube-2023.csv'
sube_2023_regresion_file_path = '../LDD-TP1/sube-2023-regresion.csv'

In [5]:
SUBE_2023 = pd.read_csv(sube_2023_file_path)
SUBE_2023_regresion = pd.read_csv(sube_2023_regresion_file_path)

#Procesamiento de Datos

##1)

###a)

Texto 1)a)

###b)

####i)

Texto 1)b)i)

####ii)


Texto 1)b)ii)

####iii)

Texto 1)b)iii)

##2)

Texto 2)

##3)

###a)

Texto 3)a)

###b)

Texto 3)b)

###c)

Texto 3)c)

#Análisis Exploratorio

##4)

Texto 4)

#Modelado

##5)

###a)

Texto 5)a)

In [6]:
SUBE_2023_regresion.head()

,DIA_TRANSPORTE,NOMBRE_EMPRESA,LINEA,AMBA,TIPO_TRANSPORTE,JURISDICCION,PROVINCIA,MUNICIPIO,CANTIDAD,DATO_PRELIMINAR
0,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,1,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,61,NO
1,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,2B,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,11,NO
2,2023-01-01,EMPRESA BATAN S.A.,BS_AS_LINEA 715M,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1707,NO
3,2023-01-01,COMPAÑIA DE TRANSPORTE VECINAL S.A.,BS_AS_LINEA_326,SI,COLECTIVO,PROVINCIAL,BUENOS AIRES,SN,438,NO
4,2023-01-01,EMPRESA DE TRANSPORTE PERALTA RAMOS SACI,BS_AS_LINEA_512,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1189,NO


In [14]:
condicion = ((SUBE_2023_regresion.AMBA == 'SI') & (SUBE_2023_regresion.TIPO_TRANSPORTE == 'COLECTIVO')) & ((SUBE_2023_regresion.JURISDICCION == 'NACIONAL') & (SUBE_2023_regresion.PROVINCIA == 'JN'))
datos_ColectivoJN = SUBE_2023_regresion[condicion]
cols = datos_ColectivoJN.LINEA.unique()  # Los nombres de las lineas de colectivo

pasajeros_por_linea = pd.DataFrame()

for col in cols:
    datos_linea = datos_ColectivoJN[datos_ColectivoJN.LINEA == col][["DIA_TRANSPORTE", "CANTIDAD"]]
    datos_linea = datos_linea.set_index("DIA_TRANSPORTE").rename(columns={"CANTIDAD": col})
    pasajeros_por_linea = pd.concat([pasajeros_por_linea, datos_linea], axis=1)

display(pasajeros_por_linea.head())


,BS_ASLINEA_123,BSAS_LINEA_002,BSAS_LINEA_009,BSAS_LINEA_010,BSAS_LINEA_015,BSAS_LINEA_017,BSAS_LINEA_019,BSAS_LINEA_020,BSAS_LINEA_021,BSAS_LINEA_022,...,LINEA 5,LINEA 50,LINEA 6,LINEA 7,LINEA 76,LINEA 8,LINEA_099,LINEA_119_AMBA,LINEA_164_AMBA,LINEA 119
DIA_TRANSPORTE,,,,,,,,,,,,,,,,,,,,,
2023-01-01,1681,5670,5644,5177,9109,7013,1604,2617,11235,2202,...,4832.0,4839,2449,2184,3797,5317,1882,1573.0,3210,NaN
2023-01-02,9440,21759,23284,21176,45269,29962,13921,11934,55611,13823,...,20333.0,16221,10486,12774,17374,23250,10322,7345.0,19362,NaN
2023-01-04,10540,24755,25405,23806,52873,33659,16172,13366,61721,16206,...,23041.0,18448,12193,13949,19860,26294,11981,7925.0,21784,NaN
2023-01-05,10408,25772,26489,24688,53436,35182,16569,12929,62510,16863,...,23928.0,18481,12888,14668,21235,27216,11787,7597.0,22972,NaN
2023-01-06,10530,26021,26458,24167,53163,35212,16459,12955,63528,16846,...,24247.0,19300,12703,14199,23472,27073,11567,7811.0,23411,NaN


###b)

Texto 5)b)

In [15]:
pasajeros_por_linea = pasajeros_por_linea.dropna(axis = 1)

###c)

Texto 5)c)

###d)

Texto 5)d)

###e)

Texto 5)e)

###f)

Texto 5)f)

###g)

Texto 5)g)